# Working with XML


## Introduction
When you retrieve information from an API, it is often stored in either an XML or a JSON format. In this lesson, we look at what XML is, the problems it solves, and provide some hands on experience working with it. 

## Objectives
You will be able to:
* Explain what XML is and why it is used
* Retrieve information from an XML file and store it in a List or DataFrame

## eXtensible Markup Language

XML (eXtensible Markup Language) is a way of taking information and making it very easy for computers to read and process it. Here is an example.

```
<people>
    <person>
        <firstname>Jane</firstname>
        <lastname>Anderson</lastname>
        <phone type="cell">111-111-1111</phone>
    </person>
    <person>
        <firstname>Joe</firstname>
        <lastname>Sonos</lastname>
        <phone type="office">111-111-1111</phone>
    </person>
    <person>
        <firstname>Alison</firstname>
        <lastname>Demming</lastname>
        <phone type="home">111-111-1111</phone>
    </person>
</people>
```
You will see that there are a set of Elements denoted by words surrounded by angle brackets (they're called tags).

You'll also see that each Element starts with a tag and ends with the same tag but with a backslash - so a person starts with the `<person>` tag and ends with the `</person>` tag. You'll also notice that Elements can be nested - so the firstname Element is nested within each of the three person Elements.

Finally, you'll notice that in addition to Elements that have values (e.g. the first firstname element has a value of `Jane`), there are also attributes. In the example above the only attribute is a `type` attribute on the phone Element, allowing us to distinguish between home, work and cell phones. The difference between a value and an attribute is that a value is contained between the opening and closing tags - e.g. `<firstname>Alison</firstname>`, whereas an attribute is written *within* the opening tag for an element - e.g. `<phone type="office">`.

**Questions:**
* **What is the second Element for each of the people?**
* **Which is the only Element in the document with an attribute?**
* **What is the closing tag for the third Element within each of the `people` Elements?**


## The Problem it Solves - Structured, Hierarchical Data Description
It's important to understand the kinds of problems that XML *(or JSON - they both solve the same problem just using different syntax)* is optimized to solve.

### Best for Structured Data
Unstructured data is something like a book or an essay - lots of words, but free form text. While unstructured data can be contained within an XML document, the strength of XML documents is in describing structured information. Structured information is where the information is broken down into types of data. So for example, I could write an unstructured essay describing a person, but I could also create a structured document describing attributes of the person such as their first name, last name, date of birth and home phone number. Such a document could be a good use case for being stored in an XML file.

### Best for Hierarchical Data
Lets say we wanted to store structured information on a list of people with their first name, last name and phone number. How could we store that? We would probably do what we've done to date. Have a csv file with column names in the first row. That would be easy to read (for either a person or a computer) and would work perfectly. So for simple structured data (that fits neatly into a simple single sheet in a spreadsheet with one type of data per column), you probably don't need to use XML *(although sometimes such information is provided using XML anyway)*.

The real strength of XML is for hierarchical data. Imagine now that we have information about a list of people. But in addition to having their first name, last name and phone number, we also have a list of one or more addresses for each person. Now how would we store and share that information?

If we know that nobody has more than (say) three addresses, we could just add a bunch of extra columns to the csv file with names like address1_street, address1_city, address1_state, address1_zip, address2_street, address2_city and so on. But what happens if some people have 20 addresses and some have none? It's a pretty inefficient way of storing the information adding twenty sets of columns, and if you suddenly find that one of the people has 21 addresses, you need to change the structure of the file and anyone working with the file will have to rewrite their scripts to handle the extra fields for that twenty first address.

With XML, it's easy. Each person can have a collection of zero or more addresses, so we might take the example above and edit it to become:

```
<people>
    <person>
        <firstname>Jane</firstname>
        <lastname>Anderson</lastname>
        <phone type="cell">111-111-1111</phone>
        <addresses></addresses>
    </person>
    <person>
        <firstname>Joe</firstname>
        <lastname>Sonos</lastname>
        <phone type="office">111-111-1111</phone>
        <addresses>
            <address type="home">
                <street>27 Magnolia Steet</street>
                <city>Maplewood</city>
                <state>NJ</state>
            </address>
            <address type="work">
                <street>4 Main Street</street>
                <city>Montclair</city>
                <state>NJ</state>
            </address>
        </addresses>
    </person>
    <person>
        <firstname>Alison</firstname>
        <lastname>Demming</lastname>
        <phone type="home">111-111-1111</phone>
        <addresses>
            <address type="home">
                <street>27 Magnolia Steet</street>
                <city>Maplewood</city>
                <state>NJ</state>
            </address>
            <address type="work">
                <street>12 Main Street</street>
                <city>Montclair</city>
                <state>NJ</state>
            </address>
        </addresses>
    </person>
</people>
```

As you can see, the XML gracefully handles both Jane who doesn't have any addresses on file, and Joe and Alison who both have two. And Joe and Alison could easily have many more addresses without changing the structure of the XML file - we'd just add some more `address` Elements within the `addresses` element that contains the collection of addresses.


## Loading XML Data

The ability to parse and process XML is so important that there is a standard library built right into Python for working with XML files. The full docs for the XML library are available [here](https://docs.python.org/3.7/library/xml.html).

For this project we'll just be using a submodule within that library - ElementTree:
https://docs.python.org/3.7/library/xml.etree.elementtree.html#module-xml.etree.ElementTree

Lets start by working with a very simple file - the address data above. The first thing we need to do is to import the ElementTree code, parse the XML file and then store those results in a variable we can have a look through.

In [1]:
import xml.etree.ElementTree as ET

tree = ET.parse('people.xml')
root = tree.getroot()
print(type(root))

<class 'xml.etree.ElementTree.Element'>


OK, now lets go look at the elements within the root of the XML document. It is a collection that is iterable, so we can iterate over the elements within the document using the `for ... in` syntax we learned earlier for looping over collections. Lets start by printing out all of the top level elements, just to see what the content looks like:

In [2]:
for child in root:
    print(child.tag, child.attrib)

person {}
person {}
person {}


OK, that's a start. So we know the root (people) is comprised of a set of person tags. Now lets see if we can dig down into those tags to start to retrieve useful information.

In [3]:
for person in root:
    print(person.tag, person.attrib)
    print('Elements:')
    for element in person:
        print('\t', element.tag, element.attrib, element.text)
    print('\n')

person {}
Elements:
	 firstname {} Jane
	 lastname {} Anderson
	 phone {'type': 'cell'} 111-111-1111
	 addresses {} None


person {}
Elements:
	 firstname {} Joe
	 lastname {} Sonos
	 phone {'type': 'office'} 111-111-1111
	 addresses {} None


person {}
Elements:
	 firstname {} Alison
	 lastname {} Demming
	 phone {'type': 'home'} 111-111-1111
	 addresses {} None




There are a few things going on in the code above. Firstly, the variable names we're using like person and element could be called anything else. The following code works equally well:

In [6]:
for duck in root:
    print(duck.tag, duck.attrib)
    print('Elements:')
    for elephant in duck:
        print('\t', elephant.tag, elephant.attrib, elephant.text)
    print('\n')

person {}
Elements:
	 firstname {} Jane
	 lastname {} Anderson
	 phone {'type': 'cell'} 111-111-1111
	 addresses {} None


person {}
Elements:
	 firstname {} Joe
	 lastname {} Sonos
	 phone {'type': 'office'} 111-111-1111
	 addresses {} None


person {}
Elements:
	 firstname {} Alison
	 lastname {} Demming
	 phone {'type': 'home'} 111-111-1111
	 addresses {} None




All Python knows is that root contains a collection of things we can iterate over, and some of those in turn contain a collection of other things that we can iterate over. What we call them doesn't matter to the computer, although it's a good idea to use meaningful names for variables so you can remember what your code was supposed to do when you come back to it later.

Secondly, if you're wondering what the `print('\t', elephant.tag, elephant.attrib, elephant.text)` does, it prints each of the comma delimited items onto a single row, so it starts by printing a tab character to indent the output (`\t`), then it prints the name of the tag, any attribute(s) the tag might contain, and finally the contents (`text`) of the tag.

There is also a convenience method .iter() that allows you to iterate through all sub generations, regardless of depth.

In [7]:
for element in root.iter():
    print(element.tag, element.attrib, element.text)

people {} 

person {} None
firstname {} Jane
lastname {} Anderson
phone {'type': 'cell'} 111-111-1111
addresses {} None
person {} None
firstname {} Joe
lastname {} Sonos
phone {'type': 'office'} 111-111-1111
addresses {} None
address {'type': 'home'} None
street {} 27 Magnolia Steet
city {} Maplewood
state {} NJ
address {'type': 'work'} None
street {} 4 Main Street
city {} Montclair
state {} NJ
person {} None
firstname {} Alison
lastname {} Demming
phone {'type': 'home'} 111-111-1111
addresses {} None
address {'type': 'home'} None
street {} 27 Magnolia Steet
city {} Maplewood
state {} NJ
address {'type': 'work'} None
street {} 12 Main Street
city {} Montclair
state {} NJ


OK, so let's try to do something useful. Let's retrieve the phone number for each of the people and put them all in a list . . .

In [8]:
phone_numbers = []
for person in root:
    for element in person:
        if element.tag == "phone":
            phone_numbers.append(element.text)
print(phone_numbers)

['111-111-1111', '111-111-1111', '111-111-1111']


Great - now lets try to get a list of addresses - lets just take the cities and make a list of them first.

In [9]:
cities = []
for person in root:
    for element in person:
        if element.tag == "addresses":
            for address in element:
                for item in address:
                    if item.tag == "city":
                        cities.append(item.text)
print(cities)

['Maplewood', 'Montclair', 'Maplewood', 'Montclair']


OK, now it's your turn! Start by iterating over the document and creating & printing a list of first names:

In [27]:
first_names = []
for person in root:
    for element in person:
        if element.tag == "firstname":
            first_names.append(element.text)
print(first_names)

['Jane', 'Joe', 'Alison']


Great, now go in and create a list of the states that the people have an address in:

In [31]:
states = []
for person in root:
    for element in person:
        if element.tag == "addresses":
            for address in element:
                for item in address:
                    if item.tag == "state":
                        states.append(item.text)
print(states)
print(set(states))

['NJ', 'NJ', 'NJ', 'NJ']
{'NJ'}


There are some other ways of working with XML files. In addition to just iterating over all of the elements under a given node, you can also find elements by name and retrieve their values by calling `.text`. So if you wanted a list of first names, you could do the following:

In [32]:
first_names = []
for person in root:
    first_names.append(person.find('firstname').text)
print(first_names)

['Jane', 'Joe', 'Alison']


Now it's your turn! Create a list of full names (hint, you'll have to concatenate the first name, a space and then the last name for every full name):

In [37]:
full_names = []
for person in root:
    full_names.append(person.find('firstname').text + " " + person.find('lastname').text)
print(full_names)

['Jane Anderson', 'Joe Sonos', 'Alison Demming']


OK, in practice, you're usually going to want to create a DataFrame from your XML file. Here is the code to create a DataFrame containing the first and last names for each person:

In [39]:
import pandas as pd

dfcols = ['firstname', 'lastname']
df = pd.DataFrame(columns=dfcols)

for person in root:
    firstname = person.find('firstname').text
    lastname = person.find('lastname').text
    df = df.append(pd.Series([firstname, lastname], index=dfcols), ignore_index=True)
df.head ()

,firstname,lastname
0,Jane,Anderson
1,Joe,Sonos
2,Alison,Demming


Great, now create a DataFrame that contains the first name, last name and phone for every person:

In [44]:
import pandas as pd

dfcols = ['firstname', 'lastname', 'phone']
df = pd.DataFrame(columns=dfcols)

for person in root:
    firstname = person.find('firstname').text
    lastname = person.find('lastname').text
    phone = person.find('phone').text
    df = df.append(pd.Series([firstname, lastname, phone], index=dfcols), ignore_index=True)
df.head ()

,firstname,lastname,phone
0,Jane,Anderson,111-111-1111
1,Joe,Sonos,111-111-1111
2,Alison,Demming,111-111-1111


## Extra Credit (1)
Sometimes you need to perform more complex transformations to create the DataFrame you want. Write some code to create a DataFrame with one row per address, containing the first name, last name and all of the address information for each address *(anyone without an address simply won't show up in the DataFrame)*.

In [69]:
import pandas as pd

dfcols = ['firstname', 'lastname', 'fulladdress']
df = pd.DataFrame(columns=dfcols)

for person in root:
    firstname = person.find('firstname').text
    lastname = person.find('lastname').text
    for address in person.find('addresses'):
        street = address.find('street').text
        city = address.find('city').text
        state = address.find('state').text
        fulladdress = (street + ' ' + city + ' ' + state)
        df = df.append(pd.Series([firstname, lastname, fulladdress], index=dfcols), ignore_index=True)
df.head ()

,firstname,lastname,fulladdress
0,Joe,Sonos,27 Magnolia Steet Maplewood NJ
1,Joe,Sonos,4 Main Street Montclair NJ
2,Alison,Demming,27 Magnolia Steet Maplewood NJ
3,Alison,Demming,12 Main Street Montclair NJ


## Extra Credit (2)
There is another file in this directory - `nyc_2001_campaign_finance.xml`. Open the file using Python, explore it using an iterator, and create a DataFrame containing the Candidate Name, Primary Pay, General Pay, Runoff Pay and Total Pay for each candidate.

In [79]:
tree2 = ET.parse('nyc_2001_campaign_finance.xml')
root2 = tree2.getroot()
print(type(root2))


<class 'xml.etree.ElementTree.Element'>


In [81]:
for child in root2:
    print(child.tag)

row


In [86]:
for row in root2:
    print(row.tag, row.attrib)
    print('Elements:')
    for element in row:
        print('\t', element.tag, element.attrib, element.text)
        for line in element:
            print('\t\t', line.tag, line.attrib, line.text)
    print('\n')

row {}
Elements:
	 row {'_id': '1', '_uuid': 'E3E9CC9F-7443-43F6-94AF-B5A0F802DBA1', '_position': '1', '_address': 'https://data.cityofnewyork.us/resource/_8dhd-zvi6/1'} None
		 candid {} CANDID
		 candname {} CANDNAME
		 officeboro {} OFFICEBORO
		 canclass {} CANCLASS
	 row {'_id': '2', '_uuid': '9D257416-581A-4C42-85CC-B6EAD9DED97F', '_position': '2', '_address': 'https://data.cityofnewyork.us/resource/_8dhd-zvi6/2'} None
		 election {} 2001
		 candid {} B4
		 candname {} Aboulafia, Sandy
		 officecd {} 5
		 officedist {} 44
		 canclass {} P
		 primarypay {} 45410.00
		 generalpay {} 0
		 runoffpay {} 0
		 totalpay {} 45410.00
	 row {'_id': '3', '_uuid': 'B80D7891-93CF-49E8-86E8-182B618E68F2', '_position': '3', '_address': 'https://data.cityofnewyork.us/resource/_8dhd-zvi6/3'} None
		 election {} 2001
		 candid {} 445
		 candname {} Adams, Jackie R
		 officecd {} 5
		 officedist {} 7
		 canclass {} P
		 primarypay {} 11073.00
		 generalpay {} 0
		 runoffpay {} 0
		 totalpay {} 11073

		 candname {} Clarke, Yvette D
		 officecd {} 5
		 officedist {} 40
		 canclass {} P
		 primarypay {} 3462.00
		 generalpay {} 74460.00
		 runoffpay {} 0
		 totalpay {} 77922.00
	 row {'_id': '45', '_uuid': '5C503029-3B4B-438B-B6D0-7E671103D2B2', '_position': '45', '_address': 'https://data.cityofnewyork.us/resource/_8dhd-zvi6/45'} None
		 election {} 2001
		 candid {} 369
		 candname {} Cohn, Steven D
		 officecd {} 5
		 officedist {} 33
		 canclass {} P
		 primarypay {} 73850.00
		 generalpay {} 73310.00
		 runoffpay {} 0
		 totalpay {} 147160.00
	 row {'_id': '46', '_uuid': '19D0A5FC-20F2-4EAF-8002-5C3F9E9582A6', '_position': '46', '_address': 'https://data.cityofnewyork.us/resource/_8dhd-zvi6/46'} None
		 election {} 2001
		 candid {} 366
		 candname {} Colon, Willie
		 officecd {} 2
		 canclass {} P
		 primarypay {} 0
		 generalpay {} 0
		 runoffpay {} 0
		 totalpay {} 0
	 row {'_id': '47', '_uuid': 'BA063336-AF01-4125-B3A5-C670217D844C', '_position': '47', '_address': 'https://d

		 election {} 2001
		 candid {} C7
		 candname {} Freed, Kathryn E
		 officecd {} 2
		 canclass {} P
		 primarypay {} 626484.00
		 generalpay {} 0
		 runoffpay {} 0
		 totalpay {} 626484.00
	 row {'_id': '86', '_uuid': 'E61A1E4A-0D1E-4496-B95B-A91C7969CA09', '_position': '86', '_address': 'https://data.cityofnewyork.us/resource/_8dhd-zvi6/86'} None
		 election {} 2001
		 candid {} 316
		 candname {} Fullard, Henrietta
		 officecd {} 5
		 officedist {} 31
		 canclass {} P
		 primarypay {} 0
		 generalpay {} 0
		 runoffpay {} 0
		 totalpay {} 0
	 row {'_id': '87', '_uuid': '5D06CFFB-6946-4FEC-9F81-9C39D7ED10BB', '_position': '87', '_address': 'https://data.cityofnewyork.us/resource/_8dhd-zvi6/87'} None
		 election {} 2001
		 candid {} 284
		 candname {} Gadson, Jeanette
		 officecd {} 4
		 officeboro {} K
		 canclass {} P
		 primarypay {} 521969.00
		 generalpay {} 0
		 runoffpay {} 0
		 totalpay {} 521969.00
	 row {'_id': '88', '_uuid': 'A2275AA6-81AA-40E9-8996-0403DEFB67E4', '_positio

		 primarypay {} 53273.00
		 generalpay {} 0
		 runoffpay {} 0
		 totalpay {} 53273.00
	 row {'_id': '125', '_uuid': '8CD56066-5E21-4213-B9A3-5FBF8AE3E4DF', '_position': '125', '_address': 'https://data.cityofnewyork.us/resource/_8dhd-zvi6/125'} None
		 election {} 2001
		 candid {} 499
		 candname {} Jeffries-El, Joseph
		 officecd {} 5
		 officedist {} 42
		 canclass {} P
		 primarypay {} 0
		 generalpay {} 0
		 runoffpay {} 0
		 totalpay {} 0
	 row {'_id': '126', '_uuid': '57411B45-06E8-42D4-BB27-EC195F515DEC', '_position': '126', '_address': 'https://data.cityofnewyork.us/resource/_8dhd-zvi6/126'} None
		 election {} 2001
		 candid {} CZ
		 candname {} Jenkins, Cynthia
		 officecd {} 5
		 officedist {} 27
		 canclass {} P
		 primarypay {} 0
		 generalpay {} 0
		 runoffpay {} 0
		 totalpay {} 0
	 row {'_id': '127', '_uuid': '15C0707E-689E-4339-89C1-D88C1A976996', '_position': '127', '_address': 'https://data.cityofnewyork.us/resource/_8dhd-zvi6/127'} None
		 election {} 2001
		 cand

		 candname {} Molinaro, James P
		 officecd {} 4
		 officeboro {} S
		 canclass {} P
		 primarypay {} 605615.00
		 generalpay {} 273841.00
		 runoffpay {} 0
		 totalpay {} 879456.00
	 row {'_id': '171', '_uuid': '1FE1893D-BBE2-400D-9633-0293B24A553B', '_position': '171', '_address': 'https://data.cityofnewyork.us/resource/_8dhd-zvi6/171'} None
		 election {} 2001
		 candid {} 272
		 candname {} Monserrate, Hiram
		 officecd {} 5
		 officedist {} 21
		 canclass {} P
		 primarypay {} 74430.00
		 generalpay {} 0
		 runoffpay {} 0
		 totalpay {} 74430.00
	 row {'_id': '172', '_uuid': 'E78D4B81-2170-4FD5-87A3-C7A867378EE8', '_position': '172', '_address': 'https://data.cityofnewyork.us/resource/_8dhd-zvi6/172'} None
		 election {} 2001
		 candid {} F5
		 candname {} Montano, Armando
		 officecd {} 5
		 officedist {} 18
		 canclass {} P
		 primarypay {} 90883.00
		 generalpay {} 0
		 runoffpay {} 0
		 totalpay {} 90883.00
	 row {'_id': '173', '_uuid': '756AA71B-B902-4189-A6D0-102470CF8725',

		 primarypay {} 62436.00
		 generalpay {} 0
		 runoffpay {} 0
		 totalpay {} 62436.00
	 row {'_id': '216', '_uuid': 'F4B86719-1C3F-4922-A715-B673FAEC3885', '_position': '216', '_address': 'https://data.cityofnewyork.us/resource/_8dhd-zvi6/216'} None
		 election {} 2001
		 candid {} EF
		 candname {} Saunders, Shirley J
		 officecd {} 5
		 officedist {} 12
		 canclass {} P
		 primarypay {} 75350.00
		 generalpay {} 0
		 runoffpay {} 0
		 totalpay {} 75350.00
	 row {'_id': '217', '_uuid': 'C9F8A1EF-08A9-4AF3-9B3B-810B08FEFCAD', '_position': '217', '_address': 'https://data.cityofnewyork.us/resource/_8dhd-zvi6/217'} None
		 election {} 2001
		 candid {} 385
		 candname {} Saunders, William J
		 officecd {} 5
		 officedist {} 35
		 canclass {} P
		 primarypay {} 42075.00
		 generalpay {} 0
		 runoffpay {} 0
		 totalpay {} 42075.00
	 row {'_id': '218', '_uuid': '779E52BF-0E2C-44D1-BACE-89B9109008FB', '_position': '218', '_address': 'https://data.cityofnewyork.us/resource/_8dhd-zvi6/218'} N

		 canclass {} P
		 primarypay {} 0
		 generalpay {} 0
		 runoffpay {} 0
		 totalpay {} 0
	 row {'_id': '262', '_uuid': '1F135761-81A0-435A-BF8E-CA7296A6114A', '_position': '262', '_address': 'https://data.cityofnewyork.us/resource/_8dhd-zvi6/262'} None
		 election {} 2001
		 candid {} 554
		 candname {} Vassos, Sandra
		 officecd {} 5
		 officedist {} 22
		 canclass {} P
		 primarypay {} 0
		 generalpay {} 68234.00
		 runoffpay {} 0
		 totalpay {} 68234.00
	 row {'_id': '263', '_uuid': '72B5E367-D882-4FCE-936D-0B9C8637B4B2', '_position': '263', '_address': 'https://data.cityofnewyork.us/resource/_8dhd-zvi6/263'} None
		 election {} 2001
		 candid {} 529
		 candname {} Vernet, Jean D
		 officecd {} 5
		 officedist {} 40
		 canclass {} P
		 primarypay {} 64932.00
		 generalpay {} 5732.00
		 runoffpay {} 0
		 totalpay {} 70664.00
	 row {'_id': '264', '_uuid': 'EA4E25C9-612B-4F3D-89D7-8F6B26D3975E', '_position': '264', '_address': 'https://data.cityofnewyork.us/resource/_8dhd-zvi6/264'} N

In [103]:
import pandas as pd
import xml.etree.ElementTree as ET

tree = ET.parse('nyc_2001_campaign_finance.xml')
root = tree.getroot()
print(type)(root)
df2cols = ['Candidate_Name', 'Primary_Pay', 'General_Pay', 'Runoff_Pay', 'Total_Pay']
df2 = pd.DataFrame(columns=df2cols)

iterrows = iter(root.find('row'))
next(iterrows)
for row in root2.find('row'):
    for element in row:
        Candidate_Name = element.find('CANDNAME').text
        Primary_Pay = line.find('primarypay').text
        General_Pay = line.find('generalpay').text
        Runoff_Pay = line.find('runoffpay').text
        Total_Pay = line.find('totalpay').text
    df2 = df.append(pd.Series([Candidate_Name, Primary_Pay, General_Pay, Runoff_Pay, Total_Pay], index=df2cols), ignore_index=True)
df2.head ()

<class 'type'>


TypeError: 'NoneType' object is not callable

## Summary

Congratulations! Now you've got some hands on experience working with importing XML files and pulling out the data you want into a DataFrame. Next up, we'll look at how to do the same with JSON files.